In [1]:
#Required library import
import cv2
import torch
from ssd_updated import build_ssd #Updated SSD file with layers and detection file
import imageio
from torch.autograd import Variable
from data import BaseTransform, VOC_CLASSES as labelmap

In [2]:
#function to detect the required objects based on the pre trained model in ssd300_mAP_77.43_v2.pth
def detect(frame, net, transform):
    #4 total transformations to the frame.
    height, width = frame.shape[:2] #here the frame object contains [h, w, c] we will exclude color as its not required for this var
    frame_t = transform(frame)[0] #the transform object will give us the array of the image of which we will only need the 1st index.
    x = torch.from_numpy(frame_t).permute(2,0,1) #Convert Numpy Array to Torch Tensor. Permute used to change color channels from RBG(0,1,2) to GRB(2,0,1) as network was trained on that sequence.
    x = Variable(x.unsqueeze(0)) #Expand the Dimensions to include the batch size
    y = net(x) #we will send the tensor x to the net
    detections = y.data #we will take the data generated from the tensor after applying the net.
    scale = torch.Tensor([width, height, width, height]) #we will create the scale object with the dimensions of the data.
    #detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]]
    for i in range(detections.size(1)):
        j=0
        while detections[0,i,j,0] >= 0.6:
            pt = (detections[0,i,j,1:] * scale).numpy() #this will be the point of the detection with scaled data to include x0...y1
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2, cv2.LINE_AA)
            j += 1
    return frame

In [3]:
#now to create the network
net = build_ssd('test')# We create an object that is our neural network ssd.
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location= lambda storage, loc: storage))

C:\Users\Aamir\Desktop\object_detection_using_ssd\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


<All keys matched successfully>

In [4]:
# We create an object of the BaseTransform class, a class that will do the required transformations so that the image can be the input of the neural network.
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0)) 

In [5]:
# Object detection on the video
reader = imageio.get_reader('test.mp4') #we open the video
fps = reader.get_meta_data()['fps'] # We get the fps frequence (frames per second).
writer = imageio.get_writer('output.mp4', fps = fps) # We create an output video with this same fps frequence.

In [6]:
for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame) # We add the next frame in the output video.
    print(i)  #We print the number of the processed frame.
    
writer.close()# We close the process that handles the creation of the output video.

C:\Users\Aamir\Desktop\object_detection_using_ssd\ssd_updated.py:108: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  21)),                # conf preds
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (640, 360) to (640, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27